In [1]:
import time

import pandas as pd

from crawler import Crawler,PropsFetchError,EmptyQueue,SubjectLog,urlkey
import bandits

In [4]:
def discover(subject,url0):
    max_size = 1000000
    log = SubjectLog(subject)
    #bandit = bandits.Genealogist(log,max_size = max_size)
    bandit = bandits.ClassyBandit(max_size = max_size)
    queue = Crawler(url0,subject,log,bandit)
    max_pages = 50 #maximal pages to consider (so including rejected)
    t0 = time.time()
    try:
        while not queue.done(max_pages,max_size) and time.time() < (t0 + 30):
            attempt = 0
            while (attempt := attempt + 1) <= 20:
                try: queue.pop().process()
                except PropsFetchError: 
                    log.nodes.append(pd.Series(name=urlkey(queue.current_url),dtype=object))
                    print('props error ',queue.current_url)
                    continue
                break
            time.sleep(0.1) # don't want to freak out wikipedia
    except EmptyQueue:
        print('Exhausted links')
    log.plot(max_size)
    return log

In [5]:
lotr = 'https://en.wikipedia.org/wiki/The_Lord_of_the_Rings'
res = discover('elves',lotr)

downloaded  96290  of  1000000
downloaded  116370  of  1000000
downloaded  215647  of  1000000


/var/folders/rt/2h5q_28967301h0wddsg5js00000gn/T/ipykernel_40564/694232086.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  log.nodes.append(pd.Series(name=urlkey(queue.current_url)),dtype=object)


TypeError: append() got an unexpected keyword argument 'dtype'